## Her skal det være en relevant overskrift.

#### Innhenter verktøy fra bibliotek
Import-stegene henter inn bibliotek med kode og funksjoner utviklet ekstern.

In [ ]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import pyspark.sql.functions as F

#### Kjører metoden read.path for å få oversikt over hvilke parquet datasett som er tilrettelagt i tilknytning veilderens lagringsområde i sky.  Oversikt blir lest inn i egen dataframe - df_datsets.
Aktuelt lagringsområde blir lagt inn som parameter (string objekt som vi definerer selv) INNDATA_PATH.

In [ ]:
INNDATA_PATH = '/felles/veiledning/pyspark/eksempler/*'
df_datasets = spark.read.path(INNDATA_PATH)
#df_datasets = spark.read.path(INNDATA_PATH+'*')

#### df_datsets skrives ut i output vindu. 

In [ ]:
df_datasets.show(100, False)

#### Avleder variabel filname fra variabel path

In [ ]:
df_datasets = df_datasets.withColumn('filename',F.substring(df_datasets.path, len(INNDATA_PATH), 30))
df_datasets.show(100, False)


#df_datasets = df_datasets.withColumn('filename',F.length(df_datasets.path))
#df_datasets = df_datasets.withColumn('filename',df_datasets.path[len(INNDATA_PATH):20])



#### Selketerer ut rader bestående av de filnavn som vi ønsker å trekke ut for videre bearbeiding. 
Viser hvordan man kan bruke liste (Phyton list) for å gjøre en slik seleksjon.

In [ ]:
filenames = ['areal', 'bnp', 'innbyggerantall/2020']
df_datasets = df_datasets.filter(df_datasets.filename.isin(filenames))
df_datasets.show()

#### Oppretter dictionaryet dict_df og leser så de tre parquet datsettene i output vindu over inn som dataframes i dictionaryet. 
Filnavnene blir keys i dictionaryet. Avslutningsvis i pargraf skrives key-navnene ut i output .

In [ ]:
dict_df = {}
for row in df_datasets.rdd.collect():
    dict_df[row.filename] = spark.read.path(row.path)
for key in dict_df:
    print(key)

### Skriver ut innhold i dataframene i dictionaryet

#### Areal

In [ ]:
dict_df['areal'].show()

In [ ]:
dict_df['areal'].count()

In [ ]:
#Tar ut oversikt over de radene i kolonnen "Land" som er duplikater
dict_df['areal'].exceptAll(dict_df['areal'].dropDuplicates(['Land'])).show()

#### BNP

In [ ]:
dict_df['bnp'].show()

#### Innbyggerantall 2020

In [ ]:
dict_df['innbyggerantall/2020'].show()

### Fjerner duplikater og kolonner vi ikke ønsker å ha med videre
Vi ønsker å sette sammen informasjon fra datasettene i dictinaryet til et nytt datasett. Dette vil vi gjøre ved å koble de sammen via Landkode variabel. Vi fjerner eventuelle dublikater på Landkode og dropper variable som vi ikke trenger/ønsker å ha med på det nye datasettet i innledende steg under.


Areal

In [ ]:
dict_df['areal'] = dict_df['areal'].dropDuplicates(['Landkode']).drop('kilde')


BNP (Legger inn variable som skal droppes fra dataframe i phyton tuple)

In [ ]:
liste_drop_kolonner = ('Land', 'Kilde', 'År')
dict_df['bnp'] = dict_df['bnp'].drop_duplicates(['Landkode']).drop(*liste_drop_kolonner)


Innbyggerantall (Legger inn variable som skal droppes fra datafrma i phyton tuple)

In [ ]:
liste_drop_kolonner = ['Land', 'Kilde', 'År']
dict_df['innbyggerantall/2020'] = dict_df['innbyggerantall/2020'].drop_duplicates(['Landkode']).drop(*liste_drop_kolonner)

### Kobler sammen datsettene og etablerer en dataframe vi skal utføre statistisk analyse på. 

Kobler først areal og bnp og oppretter dataframen df_areal_bnp. Vi legger ikke denne dataframen inn i dictionaryet. Det hadde imidlertid enkelt latt seg gjøre ved å skrive f.eks. å bytte ut df_areal_bnp med dict['areal_bnp].

In [ ]:
df_areal_bnp = dict_df['areal'].join(dict_df['bnp'], 'Landkode', how='inner')
df_areal_bnp.show()


Kobler derettet df_areal_bnp med innbyggerantall/2020 dataframen (i dictionaryet). Dermed får vi produsert den dataframen som vi ænsker å utføre en nærmere statistk analyse på. 

In [ ]:
df_sammensatt = df_areal_bnp.join(dict_df['innbyggerantall/2020'], 'Landkode', how='inner')


Skriver ut innhold i df_sammensatt i output

In [ ]:
df_sammensatt.show()

### Lagrer dataframe som parquet datasett på Dapla
Skriver pyspark dataframe til GCS bucket i parquet format
Skriver til hjemmekatalog under /user. 

In [ ]:
df_sammensatt.write\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/user/matz.ivan.faldmo/df_sammensatt_pyspark')



In [ ]:
ds = spark.read.path('/user/matz.ivan.faldmo/*')
ds.show(10, False)